# LR result

In [1]:
import argparse
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import GridSearchCV,RepeatedKFold, RepeatedStratifiedKFold, KFold, cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import os
import pickle
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, auc, roc_auc_score, recall_score, confusion_matrix, accuracy_score, f1_score, average_precision_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data = 'C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/data_final_mor.csv'
data = pd.read_csv(raw_data)
print ("Total records", data.shape[0])

Total records 11564


In [3]:
X = data.drop(['Unnamed: 0','Mortality'],axis=1).astype(int)
y = data['Mortality'].astype(int)
y = y.values.reshape(y.shape[0],1)
feature_list = X.columns
print("X:", X.shape)
print("Y:", y.shape)

X: (11564, 37)
Y: (11564, 1)


In [4]:
import joblib
LR_loaded = joblib.load('C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/PKL/modelLR1.pkl')

In [5]:
LR_loaded.best_params_

{'C': 0.1,
 'class_weight': 'balanced',
 'l1_ratio': 0.5,
 'penalty': 'l2',
 'random_state': 422,
 'solver': 'saga'}

In [6]:
print(X.shape)
print(y.shape)

(11564, 37)
(11564, 1)


In [7]:
X = X.to_numpy()

In [8]:
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=242)

test_auc = []
test_acc = []
test_ap = [] 
test_f1 = []
test_recall = []
test_precision = []

for train_index, test_index in rskf.split(X,y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  clf = LogisticRegression(**LR_loaded.best_params_)
  clf.fit(X_train, y_train)
  
  ps = clf.predict_proba(X_test)
  yp = clf.predict(X_test) 
  
  test_auc.append(roc_auc_score(y_test, ps[:,1]))
  test_acc.append(accuracy_score(y_test, yp))
  test_ap.append(average_precision_score(y_test, yp))
  test_f1.append(f1_score(y_test, yp))
  test_recall.append(recall_score(y_test, yp))
  test_precision.append(precision_score(y_test, yp))

In [9]:
LR_results = pd.DataFrame({'auc':test_auc, 'acc':test_acc, 'ap':test_ap, 'f1':test_f1, 'recall':test_recall,'precision':test_precision})
LR_results.head()

,auc,acc,ap,f1,recall,precision
0,0.837540,0.740709,0.300942,0.475524,0.839506,0.331707
1,0.844941,0.737252,0.289002,0.462898,0.808642,0.324257
2,0.846558,0.743302,0.300957,0.476190,0.828221,0.334158
3,0.833078,0.737252,0.296244,0.470383,0.828221,0.328467
4,0.853423,0.749135,0.296550,0.472727,0.802469,0.335052


In [10]:
file = 'C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/result/LR_result.csv'
LR_results.to_csv(file,index=False)